<a href="https://colab.research.google.com/github/ftnext/practice-dl-nlp/blob/master/bert_exercise/transformers_examples/20220517_example_legacy_ner_GermEval2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ref: https://github.com/huggingface/transformers/tree/v4.19.1/examples/legacy/token-classification#germeval-2014-german-ner-dataset

In [1]:
!mkdir -p /tmp/germeval2014/original

## Fetch

In [2]:
!curl -L 'https://drive.google.com/uc?export=download&id=1Jjhbal535VVz2ap4v4r_rN1UEHTdLK5P' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > /tmp/germeval2014/original/train.txt.tmp
!curl -L 'https://drive.google.com/uc?export=download&id=1ZfRcQThdtAR5PPRjIDtrVP7BtXSCUBbm' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > /tmp/germeval2014/original/dev.txt.tmp
!curl -L 'https://drive.google.com/uc?export=download&id=1u9mb7kNJHWQCWyweMDRMuTFoOHOfeBTH' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > /tmp/germeval2014/original/test.txt.tmp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 7697k  100 7697k    0     0  3396k      0  0:00:02  0:00:02 --:--:-- 85.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  706k  100  706k    0     0   516k      0  0:00:01  0:00:01 --:--:--  516k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1643k  100 1643k    0     0  1468k      0  0:00:01  0:00:01 --:--:-- 1468k


In [3]:
!wc -l /tmp/germeval2014/original/*.tmp

  43852 /tmp/germeval2014/original/dev.txt.tmp
 101598 /tmp/germeval2014/original/test.txt.tmp
 476854 /tmp/germeval2014/original/train.txt.tmp
 622304 total


## Development Environment

In [4]:
!git clone https://github.com/huggingface/transformers.git
!cd transformers && git checkout v4.19.1&& pip install .

Cloning into 'transformers'...
remote: Enumerating objects: 95488, done.
remote: Counting objects: 100% (314/314), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 95488 (delta 147), reused 174 (delta 74), pack-reused 95174
Receiving objects: 100% (95488/95488), 88.73 MiB | 28.69 MiB/s, done.
Resolving deltas: 100% (70138/70138), done.
Note: checking out 'v4.19.1'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a1651b518 Release: v4.19.1
Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory.

In [5]:
!pip install seqeval filelock conllu

     |████████████████████████████████| 43 kB 1.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=6c299f61043c31217275785b62ec4bb6fed2b5de1468b471628ac2fb1b7e2167
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [6]:
!pip list | grep torch

torch                         1.11.0+cu113
torchaudio                    0.11.0+cu113
torchsummary                  1.5.1
torchtext                     0.12.0
torchvision                   0.12.0+cu113


## Preprocess

In [7]:
!cd transformers/examples/legacy/token-classification && python scripts/preprocess.py \
  /tmp/germeval2014/original/train.txt.tmp bert-base-multilingual-cased 128 > /tmp/germeval2014/train.txt

Downloading: 100% 29.0/29.0 [00:00<00:00, 32.2kB/s]
Downloading: 100% 625/625 [00:00<00:00, 754kB/s]
Downloading: 100% 972k/972k [00:00<00:00, 5.45MB/s]
Downloading: 100% 1.87M/1.87M [00:00<00:00, 8.08MB/s]


In [8]:
!cd transformers/examples/legacy/token-classification && python scripts/preprocess.py \
  /tmp/germeval2014/original/dev.txt.tmp bert-base-multilingual-cased 128 > /tmp/germeval2014/dev.txt

In [9]:
!cd transformers/examples/legacy/token-classification && python scripts/preprocess.py \
  /tmp/germeval2014/original/test.txt.tmp bert-base-multilingual-cased 128 > /tmp/germeval2014/test.txt

In [10]:
!wc -l /tmp/germeval2014/*.txt

  43851 /tmp/germeval2014/dev.txt
 101597 /tmp/germeval2014/test.txt
 476852 /tmp/germeval2014/train.txt
 622300 total


In [11]:
!cd /tmp/germeval2014 && cat train.txt dev.txt test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [12]:
!cat /tmp/germeval2014/labels.txt

B-LOC
B-LOCderiv
B-LOCpart
B-ORG
B-ORGderiv
B-ORGpart
B-OTH
B-OTHderiv
B-OTHpart
B-PER
B-PERderiv
B-PERpart
I-LOC
I-LOCderiv
I-LOCpart
I-ORG
I-ORGderiv
I-ORGpart
I-OTH
I-OTHderiv
I-OTHpart
I-PER
I-PERderiv
I-PERpart
O


In [13]:
!wc -l /tmp/germeval2014/labels.txt

25 /tmp/germeval2014/labels.txt


## Train

In [14]:
!cd transformers/examples/legacy/token-classification && python run_ner.py \
--data_dir /tmp/germeval2014 \
--labels /tmp/germeval2014/labels.txt \
--model_name_or_path bert-base-multilingual-cased \
--output_dir /tmp/ner-germeval2014 \
--max_seq_length 128 \
--num_train_epochs 3 \
--per_device_train_batch_size 32 \
--save_steps 750 \
--seed 1 \
--do_train \
--do_eval \
--do_predict

05/17/2022 12:49:20 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
05/17/2022 12:49:20 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=

In [15]:
!ls -l /tmp/ner-germeval2014/

total 694440
drwxr-xr-x 2 root root      4096 May 17 13:07 checkpoint-1500
drwxr-xr-x 2 root root      4096 May 17 13:16 checkpoint-2250
drwxr-xr-x 2 root root      4096 May 17 12:58 checkpoint-750
-rw-r--r-- 1 root root      1899 May 17 13:16 config.json
-rw-r--r-- 1 root root       269 May 17 13:16 eval_results.txt
-rw-r--r-- 1 root root 709198705 May 17 13:16 pytorch_model.bin
drwxr-xr-x 3 root root      4096 May 17 12:50 runs
-rw-r--r-- 1 root root       112 May 17 13:16 special_tokens_map.json
-rw-r--r-- 1 root root    859809 May 17 13:17 test_predictions.txt
-rw-r--r-- 1 root root       256 May 17 13:17 test_results.txt
-rw-r--r-- 1 root root       381 May 17 13:16 tokenizer_config.json
-rw-r--r-- 1 root root      3183 May 17 13:16 training_args.bin
-rw-r--r-- 1 root root    995526 May 17 13:16 vocab.txt


In [16]:
!cat /tmp/ner-germeval2014/eval_results.txt

eval_loss = 0.07414335757493973
eval_accuracy_score = 0.981393450494574
eval_precision = 0.8445318084345962
eval_recall = 0.8836948391922214
eval_f1 = 0.8636695906432749
eval_runtime = 22.4736
eval_samples_per_second = 97.893
eval_steps_per_second = 12.237
epoch = 3.0


In [17]:
!cat /tmp/ner-germeval2014/test_results.txt

test_loss = 0.0697232037782669
test_accuracy_score = 0.9832328131152978
test_precision = 0.8598070739549839
test_recall = 0.8656523146649401
test_f1 = 0.862719793515083
test_runtime = 54.1932
test_samples_per_second = 94.108
test_steps_per_second = 11.773
